# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 06**: Big Data Pipeline for Netflix data

**Fecha**: 7 de febrero de 2025

**Nombre del Estudiante**: JJAE

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://843542683a54:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 14:33:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Netflix data

In [3]:
from JJAE.spark_utils import SparkUtils
netflix_schema = SparkUtils.generate_schema([
    ("show_id", "string"),
    ("type", "string"),
    ("title", "string"),
    ("director", "string"),
    ("country", "string"),
    ("date_added", "string"),
    ("release_year", "integer"),
    ("dating", "string"),
    ("duration", "string"),
    ("listen_in", "string")
    ])

netflix_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

netflix_df.printSchema()

netflix_df.show(5, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- dating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listen_in: string (nullable = true)



+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|dating|duration|listen_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead            |Kirsten Johnson|United States|9/25/2021 |2020        |PG-13 |90 min  |Documentaries                                                |
|s3     |TV Show|Ganglands                       |Julien Leclercq|France       |9/24/2021 |2021        |TV-MA |1 Season|Crime TV Shows, International TV Shows, TV Action & Adventure|
|s6     |TV Show|Midnight Mass                   |Mike Flanagan  |United States|9/24/

In [8]:
netflix_df.summary().show()

25/03/07 14:36:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----------------+-------------+---------------------------------+------------+--------+----------+------------------+--------------------+--------+------------------+
|summary|         show_id|         type|                            title|    director| country|date_added|      release_year|              dating|duration|         listen_in|
+-------+----------------+-------------+---------------------------------+------------+--------+----------+------------------+--------------------+--------+------------------+
|  count|            8791|         8791|                             8791|        8790|    8790|      8790|              8789|                8790|    8789|              8789|
|   mean|            NULL|         NULL|               1124.7692307692307|        NULL|  1944.0|      NULL|2014.1911480259416|                NULL|    NULL|              NULL|
| stddev|            NULL|         NULL|               1042.1800991068478|        NULL|    NULL|      NULL|  8.794154289